# Patchy particles angular potential 

Implempentation of two freely diffusing patchy particles (rigid bodies) with two patches and pair interactions given by the patchy particle angular potential defined in the main code. The main difference between this potential and the patchy particle potential is the addition of angular dependence. Each binding between patches will have two possible preferred relative orientations. This yields two possible conformations per binding between patches (one per metastable orientation). 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2
import msmrd2.tools.quaternions as quats
import msmrd2.visualization as msmrdvis
from msmrd2.potentials import patchyParticleAngular
from msmrd2.integrators import overdampedLangevin as odLangevin

In [ ]:
# Define arbitrarily large list of patchy particles
numparticles = 2
boxsize = 3 
D = 1.0
Drot = 1.0
pyPartlist = [] 
positionList = [np.array([-1.0,0,0]), np.array([0.,0.,0.]), np.array([1.0,0.,0.]), 
                np.array([2.0,0.,0.]), np.array([3.0,0.,0.])]
orientationList = [ np.array([1.,0.,0.,0.]), np.array([np.cos(np.pi/2),0.,0.,np.sin(np.pi/2)])]

for i in range(numparticles):
    
    overlap = True
    while overlap:
        position = np.array([boxsize*np.random.rand()-0.5*boxsize,
                             boxsize*np.random.rand()-0.5*boxsize,
                             boxsize*np.random.rand()-0.5*boxsize])
        overlap = False     
        for j in range(len(pyPartlist)):
            if np.linalg.norm(position - pyPartlist[j].position) < 2.0:
                overlap = True
                continue
                
    orientation = np.array([np.random.rand(),np.random.rand(),np.random.rand(),np.random.rand()])
    orientation = orientation/np.linalg.norm(orientation)
    part = msmrd2.particle(D, Drot, position, orientation)
    pyPartlist.append(part)

In [ ]:
# Create list of particles that can be read from msmrd
# Note the particles in this list will be independent from the python list.
partlist = msmrd2.integrators.particleList(pyPartlist)

In [ ]:
# Over-damped Langevin integrator definition
dt = 0.00001 #0.000005
seed = -1 # Negative seed, uses random device as seed
bodytype = 'rigidbody' # three rotational degrees of freedom
integrator = odLangevin(dt, seed, bodytype) 
#integrator.setKbT(0.1) # possible to change KbT (Boltzmann constant and temperature)

In [ ]:
# Define boundary (choose either spherical or box)
boxBoundary = msmrd2.box(boxsize,boxsize,boxsize,'reflective')
integrator.setBoundary(boxBoundary)

In [ ]:
# Define Patchy Particle potential
sigma = 1.0
strength = 160 #200.0
angularStrength = 20 #200.0
angleDiff = 3*np.pi/5.0
patch1 = np.array([np.cos(angleDiff/2),np.sin(angleDiff/2),0.])
patch2 = np.array([np.cos(-angleDiff/2),np.sin(-angleDiff/2),0.])
patchesCoordinates = [patch1, patch2]
potentialPatchyParticleAngular = patchyParticleAngular(sigma, strength, angularStrength, patchesCoordinates)
integrator.setPairPotential(potentialPatchyParticleAngular)

In [ ]:
# Integrate particle list and print only positions 
timesteps = 10000000 #20000000
stride = 500 #250 #1000
datafile  = open('../../data/vmd/patchyParticlesAngular.xyz', 'w')
for i in range(timesteps):
    if i%stride == 0:
        datafile.write(str(3*len(partlist)) + '\n')
        datafile.write(str(0) + '\n')
    for j, part in enumerate(partlist):
        if i%stride == 0:
            v0 = part.position
            v1 = v0 + 0.5*sigma*quats.rotateVec(patchesCoordinates[0], part.orientation)
            v2 = v0 + 0.5*sigma*quats.rotateVec(patchesCoordinates[1], part.orientation)
            datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
            datafile.write('type_1' + ' ' + ' '.join(map(str, v1)) + '\n')
            datafile.write('type_1' + ' ' + ' '.join(map(str, v2)) + '\n')
    integrator.integrate(partlist)
    if i%10000 == 0:
        print("Percentage complete: ", 100*i/timesteps, "%", end="\r")
datafile.close()
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_patchyParticles(numparticles = numparticles, 
                                     outfname = "patchyParticlesAngular", 
                                     tclfname = "../../data/vmd/patchyParticlesAngular_2vmd.tcl")
print("Percentage complete: ", 100, " %")

To load the movie go to /data/vmd and run in a terminal "vmd -e patchyPacrticlesAngular_2vmd.tcl".

## In case VMD is not desired

In case VMD output is not desired, below we simply integrate the model and show the output directly.

In [ ]:
# Integrate particle list and print only positions of first and last particle in list
timesteps = 10
print('{:<10s}{:<15s}{:<40s}{:<40s}'.format("Iteration", "Time", "Position 1", "Position 2"))
for i in range(timesteps):
    print('{:<10d}{:<15f}{:<40s}{:<50s}'.format(i, integrator.clock, str(partlist[0].position), str(partlist[1].position)))
    integrator.integrate(partlist)